## Read in processed data from the Boston master merger notebook

In [1]:
# This will take several mintues to run. Kick back and relax. Or this is a good time to get a cup of coffee.
%run Boston_311_Master_Merger.ipynb

Holy cow! I hope you got coffee during the runtime: 0:13:46.559900


In [2]:
# Examine the shape the master dataframe from the merged ipynb
print('Number of Rows: ', master.shape[0])
print('Number of Columns: ', master.shape[1])

Number of Rows:  1864762
Number of Columns:  97


In [3]:
# Explore the fields and values from the master dataframe to determine what fields we will need for ML
pd.set_option('display.max_columns', None)
master.head()

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,type,queue,department,submittedphoto,closedphoto,location,fire_district,pwd_district,city_council_district,police_district,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,source,date,avgprcp,geometry,index_right,field concept,State/US-Abbreviation (USPS),Summary Level,Geographic Record Identifier,Geographic Code Identifier,Region,Division,State (FIPS),County (FIPS),County Subdivision (FIPS),Census Tract,Block Group,GEOID 2020 block group,Total_P1,White alone,Black or African American alone,Hispanic or Latino,Asian alone,American Indian and Alaska Native alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Population of two or more races:,Total_P2,White alone,Black or African American alone,Hispanic or Latino,Asian alone,American Indian and Alaska Native alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Population of two or more races:,Total_P3,Institutionalized population:,Correctional facilities for adults,Juvenile facilities,Nursing facilities/Skilled-nursing facilities,Other institutional facilities,Noninstitutionalized population:,College/University student housing,Military quarters,Other noninstitutional facilities,Total_P4,Occupied,Vacant,GRI,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,BLKGRPCE20,GEOID20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,Shape_STAr,Shape_STLe,ucr_one,ucr_two,total_ucr
0,101001409422,2015-06-15 00:59:59,NaN,2015-06-15 08:33:17,ONTIME,Closed,Case Closed Case Noted The City of Boston doe...,Schedule Bulk Item Pickup,Public Works Department,Sanitation,Schedule a Bulk Item Pickup SS,PWDx_Schedule a Bulk Item Pickup,PWDx,NaN,NaN,10 Varney St Jamaica Plain MA 02130,12,02,6,E13,Jamaica Plain,10,Ward 19,1907,10 Varney St,2130.0,42.2956,-71.1147,Self Service,2015-06-15,102.0,POINT (-71.11470 42.29560),456.0,PLST,MA,150,1500000US250251101051,2.50E+11,1,1,25,25,NaN,110105,1,2.50E+11,2120,1233,158,281,302,1,2,23,120,1873,1126,146,226,270,1,2,15,87,3,0,0,0,0,0,3,0,0,3,1160,981,179,250251101051,474.0,475.0,25,025,110105,1,250251101051,Block Group 1,G5030,S,348074.0,0.0,+42.2988673,-071.1114074,3.746367e+06,9846.512298,101.0,436.0,537.0
1,101001409423,2015-06-15 02:42:00,NaN,NaT,ONTIME,Open,,Animal Generic Request,Animal Control,Animal Issues,Animal Generic Request,INFO01_GenericeFormforOtherServiceRequestTypes,INFO,NaN,NaN,49 Wellington Hill St Mattapan MA 02126,8,07,4,B3,Greater Mattapan,9,Ward 14,1408,49 Wellington Hill St,2126.0,42.2808,-71.0950,Constituent Call,2015-06-15,102.0,POINT (-71.09500 42.28080),447.0,PLST,MA,150,1500000US250251011011,2.50E+11,1,1,25,25,NaN,101101,1,2.50E+11,1033,25,755,177,2,4,0,14,56,783,19,602,99,1,4,0,11,47,0,0,0,0,0,0,0,0,0,0,358,323,35,250251011011,555.0,556.0,25,025,101101,1,250251011011,Block Group 1,G5030,S,142066.0,0.0,+42.2824133,-071.0941760,1.529082e+06,4975.264383,83.0,294.0,377.0
2,101001409424,2015-06-15 02:45:00,2015-06-18 08:30:00,NaT,OVERDUE,Open,,Work Hours-Loud Noise Complaints,Public Works Department,Highway Maintenance,Work Hours-Loud Noise Complaints,INFO_General Comments Complaints Compliments,INFO,NaN,NaN,116 Williams St Jamaica Plain MA 02130,9,02,6,E13,Jamaica Plain,11,Ward 11,1107,116 Williams St,2130.0,42.3047,-71.1053,Constituent Call,2015-06-15,102.0,POINT (-71.10530 42.30470),488.0,PLST,MA,150,1500000US250251202012,2.50E+11,1,1,25,25,NaN,120201,2,2.50E+11,1872,861,300,509,75,8,2,17,100,1514,732,242,390,67,6,2,10,65,7,0,0,0,0,0,7,0,0,7,905,867,38,250251202012,353.0,354.0,25,025,120201,2,250251202012,Block Group 2,G5030,S,324751.0,0.0,+42.3068650,-071.1058089,3.495318e+06,7723.528390,116.0,444.0,560.0
3,101001409425,2015-06-15 04:29:00,2015-06-16 08:30:00,2015-06-16 12:24:05,OVERDUE,Closed,Case Closed. Closed date : 2015-06-16 12:24:05...,Request for Pothole Repair,Public Works Department,Highway Maintenan

## Select the fields that will potentially be used for further feature selection and later modeling

In [4]:
# Define the fields we will use for ML
cols = ['case_enquiry_id','date','ontime','case_status','subject','reason','avgprcp','Total_P1','GEOID20','ucr_one','ucr_two','total_ucr']
master = master[cols]

In [5]:
master.shape

(1864762, 12)

In [6]:
master.head()

,case_enquiry_id,date,ontime,case_status,subject,reason,avgprcp,Total_P1,GEOID20,ucr_one,ucr_two,total_ucr
0,101001409422,2015-06-15,ONTIME,Closed,Public Works Department,Sanitation,102.0,2120,250251101051,101.0,436.0,537.0
1,101001409423,2015-06-15,ONTIME,Open,Animal Control,Animal Issues,102.0,1033,250251011011,83.0,294.0,377.0
2,101001409424,2015-06-15,OVERDUE,Open,Public Works Department,Highway Maintenance,102.0,1872,250251202012,116.0,444.0,560.0
3,101001409425,2015-06-15,OVERDUE,Closed,Public Works Department,Highway Maintenance,102.0,610,250250701042,617.0,1644.0,2261.0
4,101001409429,2015-06-15,ONTIME,Open,Animal Control,Animal Issues,102.0,1354,250250920001,156.0,620.0,776.0


## Date enrichment

In [7]:
# Breakout day, month, and year from the date field
master['day'] = master['date'].dt.day
master['month'] = master['date'].dt.month
master['year'] = master['date'].dt.year

## Encoding

In [18]:
# See which fields contain categorical vs numerical values
master.dtypes

case_enquiry_id             int64
date               datetime64[ns]
ontime                     object
case_status                object
subject                    object
reason                     object
avgprcp                   float64
Total_P1                   object
GEOID20                    object
ucr_one                   float64
ucr_two                   float64
total_ucr                 float64
day                         int64
month                       int64
year                        int64
dtype: object

In [40]:
# Encode categorical features
master_encoded = pd.get_dummies(data=master, columns=['reason', 'subject', 'ontime'])

master_encoded['closed'] = master_encoded['case_status'].apply(lambda x: 1 if x=='Closed' else 0)

master_encoded = master_encoded.drop('case_status', axis=1)

master_encoded['Total_P1'] = pd.to_numeric(master_encoded.Total_P1, errors='coerce')

master_encoded['GEOID20'] = pd.to_numeric(master_encoded.GEOID20, errors='coerce')


In [41]:
master_encoded.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1864762 entries, 0 to 1864761
Data columns (total 78 columns):
 #   Column                                     Dtype         
---  ------                                     -----         
 0   case_enquiry_id                            int64         
 1   date                                       datetime64[ns]
 2   avgprcp                                    float64       
 3   Total_P1                                   float64       
 4   GEOID20                                    float64       
 5   ucr_one                                    float64       
 6   ucr_two                                    float64       
 7   total_ucr                                  float64       
 8   day                                        int64         
 9   month                                      int64         
 10  year                                       int64         
 11  reason_Abandoned Bicycle                   uint8         
 12  

## Feature Analysis

In [ ]:
# Pearson's Correlation

In [ ]:
import matplotlib.pyplot as plt # visualization
import seaborn as sns

sns.set_theme(style="darkgrid")

# Compute a correlation matrix
corr = master_encoded.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=False, mask = mask, cmap=cmap);


In [ ]:
# we clearly have class imbalance
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_theme(style="darkgrid")

status_count_fig = master_encoded['closed'].value_counts().plot(kind='bar')
status_count_fig.bar_label(status_count_fig.containers[0], fmt='%d' );



## Model Selection

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve
from sklearn.model_selection import train_test_split



In [ ]:
from sklearn.utils import resample

closed=master_encoded[master_encoded.closed==1]
open=master_encoded[master_encoded.closed==0]

# upsample minority
open_upsampled = resample(open
                            , replace=True # sample with replacement 
                            , n_samples=len(closed) # match number in majority class
                          , random_state=12022022) # set seed

# combine majority and upsampled minority
upsampled = pd.concat([closed, open_upsampled])

# check new class counts
upsampled.closed.value_counts()

In [ ]:
y = upsampled['closed']

X = upsampled.drop['closed']
